### 01 - Data Loading, Visualization, and Preprocessing

This notebook serves as the initial data exploration and validation step. The goal is to:

1. Confirm successful reading of FastMRI HDF5 files and k-space extraction.
2. Inspect the structure and distribution of the fastmri+ clinical labels.
3. Prototype the entire data flow: Generate the undersampling mask, apply it to the k-space, and visualize the resulting Zero-Filled IFFT (Artifact Baseline / Pipeline A U-Net Input) against the Fully Sampled IFFT (Ground Truth).